## Compute forecast metrics using item-level backtests

<b>DC Bikeshare Rentals data</b>

Our goal is to predict the number of DC Bikeshare rentals in the next 24 hours for each of 467 rental locations.  To do this, we used Amazon Forecast (with Target Time Series, i.e. historical demand only) to create baseline forecasts with 1 hour frequency and 1 week forecast horizon.  For this demo, we did not use any metadata or custom features related data, which is usually done as a next step to get improved accuracy.
<ul>
    <li>See <a href="https://aws.amazon.com/blogs/machine-learning/measuring-forecast-model-accuracy-to-optimize-your-business-objectives-with-amazon-forecast/", target='_blank'>blog post for screens how the forecast was created.</a></li>
<li>Original data source: <a href="https://www.capitalbikeshare.com/system-data", target='_blank'> https://www.capitalbikeshare.com/system-data</a> </li>
    </ul>  

This notebook will show how you can validate your Predictor (validate model train) before deploying the Predictor to make Forecasts.  Generally, in machine learning, you want to validate a model on train/validation data before deciding to deploy a model to make inferences in production.  In the overall Amazon Forecast workflow, this notebook covers <i>step 6. Inspect the model using the backtest window forecasts, see context below.</i>
<br><br>


<b>Overall process for using Amazon Forecast:</b>

!["Amazon Forecast overview workflow"](images/forecast_steps_overview.png "Amazon Forecast overview workflow")

<ol>
    <li>Prepare your data and save up to 3 separate .csv files.  In Forecast there are  3 types of Datasets (Target, Related, and Meta data). <br>
        <ul>
        <li>The Target Time Series is required, it is the historical values of what you're trying to predict, i.e. historical y-values or target values.  The others provide additional context with certain algorithms. <br>
        </ul><br>
    <li>Per dataset, you will specify the schema. </li>
    <li>Per dataset, create a Data Import Job. Give the S3 location where data will be read from. </li>
    <li>Create a Dataset Group.  This is a container that groups together your models, data they are trained on, and forecasts.  <br>
        <ul>
            <li>Having this grouping is convenient if in the future, you want to look up artifacts how you ran a particular forecast. </li>
        <li>The Dataset Group is also how you can run inferences (forecasts) in the future, without retraining, just by importing new data, by creating a new Data Import job.  </li>
        </ul><br>
    <li>Train a model.  Amazon Forecast offers AutoML to do this process for you, but you can also select a particular algorithm (6 built-in algorithms).  AutoML will do Hyper Parameter Optimization(HPO) to determine the most performant values automatically, or you can select your own values.</li>
    <b><li>Inspect the model using the backtest window forecasts. This notebook focuses on this step.</li>
    <ul>
    <li>We will use the built-in feature of Amazon Forecast that exports backtest window forecasts together with actuals, for Predictor analysis.  </li>
    <li>Predictor evaluation is recommended to make an informed decision whether to deploy the current Predictor to make Forecasts, or whether to fix something in the data setup and train a new Predictor. </li></b>
    </ul><br>
    <li>Deploy the model (or create a forecast).  Here you are deploying your model so you can use it to do inferences (or generate forecasts).</li>
    <li>Query and visualize the Forecast (available in console UI). Spot-check actual values and forecasted values at different quantiles in the console for a particular itemID. The visualization feature in the console is basic, not user-interactive.  For more advanced visualizations, consider exporting your forecasts to an S3 location and point your BI tool (e.g. Tableau or Quicksight) to that data.</li>
    </ol>
<br>


<b>Table Of Contents for task of inspecting the model using the backtest window forecasts</b>
* [Set up and install libraries](#setup)
* [Export predictor backtests](#export)
* [Assemble and read predictor backtest files](#read)
* [Demo using item-level forecast files](#demo)
* [Visualize backtest window accuracy](#visualizations)
* [Calculate custom MAPE](#mape)
* [Cleanup](#cleanup)

<br>

## Set up  <a class="anchor" id="setup"></a>
Import and install Python and aws libraries


In [1]:
import sys
import os
import glob
import shutil
import time
import datetime
from datetime import timedelta
import random

import pandas as pd
print('pandas: {}'.format(pd.__version__))
# display all columns wide
pd.set_option('display.max_columns', None)
# display all rows long
pd.set_option('display.max_rows', None)
# display horizontal scrollbar for wide columns
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', 5000)
#turn off scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import numpy as np
print('numpy: {}'.format(np.__version__))
import matplotlib.pyplot as plt
%matplotlib inline 


# Python library for AWS APIs
import boto3

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util

pandas: 1.0.5
numpy: 1.19.1


In [2]:
#########
# Function to concat .part files
#########

def read_backtest_predictions(BUCKET_NAME, s3_path):
    """Read predictor backtest predictions export files
       Inputs: 
           BUCKET_NAME = S3 bucket name
           s3_path = S3 path to Predictor.part files
                         , everything after "s3://BUCKET_NAME/" in S3 URI path to your .part files
       Return: Pandas dataframe with all .part files concatenated row-wise
    """
    # set s3 path
    s3 = boto3.resource('s3')
    s3_bucket = boto3.resource('s3').Bucket(BUCKET_NAME)
    s3_depth = s3_path.split("/")
    s3_depth = len(s3_depth) - 1
    
    # set local path
    local_write_path = "visualize"
    !mkdir -p $local_write_path
    !rm -rf $local_write_path/*
    
    # concat part files
    part_filename = ""
    part_files = list(s3_bucket.objects.filter(Prefix=s3_path))
    print(f"Number .part files found: {len(part_files)}")
    for file in part_files:
        # There will be a collection of CSVs if the forecast is large, modify this to go get them all
        if "csv" in file.key:
            part_filename = file.key.split('/')[s3_depth]
            window_object = s3.Object(BUCKET_NAME, file.key)
            file_size = window_object.content_length
            if file_size > 0:
                s3.Bucket(BUCKET_NAME).download_file(file.key, local_write_path+"/"+part_filename)
        
    # Read from local dir and combine all the part files
    temp_dfs = []
    for entry in os.listdir(local_write_path):
        if os.path.isfile(os.path.join(local_write_path, entry)):
            df = pd.read_csv(os.path.join(local_write_path, entry), index_col=None, header=0)
            temp_dfs.append(df)

    # Return assembled .part files as pandas Dataframe
    fcst_df = pd.concat(temp_dfs, axis=0, ignore_index=True, sort=False)
    return fcst_df


#########
# Functions to calculate item velocity and classify items a "slow" or "fast"
#########

def get_velocity_per_item(df, timestamp_col, item_id_col="item_id"):
    """Calculate item velocity as item demand per hour.  
    """
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], format='%Y-%m-%d %H:%M:%S')
    
    max_time_df = df.groupby([item_id_col], as_index=False).max()[[item_id_col, timestamp_col]]
    max_time_df.columns = [item_id_col, 'max_time']
    
    min_time_df = df.groupby([item_id_col], as_index=False).min()[[item_id_col, timestamp_col]]
    min_time_df.columns = [item_id_col, 'min_time']
    
    df = df.merge(right=max_time_df, on=item_id_col)
    df = df.merge(right=min_time_df, on=item_id_col)
    
    df['time_span'] = df['max_time'] - df['min_time']
    df['time_span'] = df['time_span'].apply(lambda x: x.seconds / 3600 + 1) # add 1 to include start datetime and end datetime
    df = df.groupby([item_id_col], as_index=False).agg({'time_span':'mean', 'target_value':'sum'})
    df['velocity'] = df['target_value'] / df['time_span']
    return df


def get_fast_slow_moving_items_all(gt_df
                                   , timestamp_col
                                   , target_value_col
                                   , item_id_col="item_id"):
    """Calculate mean velocity over all items as "criteria".
       Assign each item into category "fast" or "slow" depending on whether its velocity > criteria.
    """
    gt_df_velocity = gt_df[[item_id_col, timestamp_col, target_value_col]].copy().reset_index(drop=True)
    gt_df_velocity = get_velocity_per_item(gt_df_velocity, timestamp_col, item_id_col)
    criteria = gt_df_velocity['velocity'].mean()
    gt_df_velocity['fast_moving'] = gt_df_velocity['velocity'] > criteria
    print('average velocity of all items:', criteria)
    
    fast_moving_items = gt_df_velocity[gt_df_velocity['fast_moving'] == True][item_id_col].to_list()
    slow_moving_items = gt_df_velocity[gt_df_velocity['fast_moving'] == False][item_id_col].to_list()
    return fast_moving_items, slow_moving_items


###########
# Define custom metrics
###########

def truncate_negatives_to_zero(the_df, target_value_col, quantile_cols):
    """In case you are expecting positive numbers for actuals and predictions,
       round negative values up to zero.
       
       Be careful that this is acceptable treatment of negatives for your use case.
    """
    
    df = the_df.copy()
    
    for q in quantile_cols:
        num_neg_predictions = df[q].lt(0).sum()
        print(f"Num negative {q} predictors: {num_neg_predictions}")

        # replace
        df[q] = df[q].mask(df[q] < 0, 0)

        # check you did the right thing
        num_neg_predictions = df[q].lt(0).sum()
        print(f"Num negative {q} predictors: {num_neg_predictions}")

    # truncate negative actuals
    num_neg_actuals = df[target_value_col].lt(0).sum()
    print(f"Num negative actuals: {num_neg_actuals}")

    # replace
    df[target_value_col] = df[target_value_col].mask(df[target_value_col] < 0, 0)

    # check you did the right thing
    num_neg_actuals = df[target_value_col].lt(0).sum()
    print(f"Num negative actuals: {num_neg_actuals}")
    
    return df

       
def calc_mape(target, forecast):
    """Calculates custom mape for a specific quantile and window with formula:
            sum(| |predicted| - |actual| | / |actual|)
       Input: single numbers for target and forecast
       Output: mape = floating point number
    """
    denominator = np.abs(target)
    flag = denominator <= 1e-8

    mape = np.mean(
        (np.abs( np.abs(target) - np.abs(forecast)) * (1.0 - flag)) / (denominator + flag)
    )
    return mape


def quantile_loss(actual, pred, quantile):
    """Calculate weighted quantile loss for a specific quantile and window
       Input: single numbers for actual and forecast
       Output:  wql = floating point number
    """
    denom = sum(np.abs(actual))
    num = sum([(1-quantile) * abs(y_hat-y) if y_hat > y
               else quantile * abs(y_hat-y) for y_hat, y in zip(pred, actual)])
    if denom != 0:
        return 2 * num / denom
    else:
        return None



In order to run Amazon Forecast, you'll need an AWS account.  
<b>Make sure you can log in to: https://console.aws.amazon.com/.  </b>  Then read each cell carefully and execute the cells in this notebook.
<br>
<br>

<b>Configure the S3 bucket name and region name for this lesson.</b>

- If you don't have an S3 bucket, create it first on S3.
- Although we have set the region to us-west-2 as a default value below, you can choose any of the regions that the service is available in.

In [3]:

## Get user inputs for S3 bucket name and region

# TODO: put back default value or user override value
default_bucket = "christy-forecast"  #default bike-demo
BUCKET_NAME = input("S3 bucket name [enter to accept default]: ") or default_bucket
default_region = 'us-west-2'
REGION = input(f"region [enter to accept default]: {default_region}") or default_region 


S3 bucket name [enter to accept default]: 
region [enter to accept default]: us-west-2


The next part of the setup process is to validate that your account can communicate with Amazon Forecast

In [4]:
# Initialize forecast session
session = boto3.Session(region_name=REGION) 
forecast = session.client(service_name='forecast') #Amazon Forecast Service api session
# forecastquery = session.client(service_name='forecastquery') #Amazon Forecast Query api session - not used here


The last part of the setup process is to create an AWS Role with Forecast and S3 permissions

In [5]:
# Create the role to provide to Amazon Forecast.
# role_name = "ForecastNotebookRole"
# print(f"Creating Role {role_name} ...")
# role_arn = util.get_or_create_iam_role( role_name = role_name )

import sagemaker
from sagemaker import get_execution_role
default_role = get_execution_role()

print(f"Success! Created role arn = {role_arn}")

Couldn't call 'get_role' to get Role ARN from role name christy to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::625299737718:user/christy, therefore it cannot be used as a SageMaker execution role

### Export predictor backtests <a class="anchor" id="export"></a>

"Backtesting" is a cross-validation technique for time series that uses multiple train/test splits that keep time order of the data.  Using multiple train-test splits (i.e. more than 1 backtest window) will result in more models being trained, and in turn, a more robust estimate how the model (chosen algorithm and hyperameters) will perform on unseen data.
<a href="https://docs.aws.amazon.com/forecast/latest/dg/metrics.html#backtesting, target='_blank' ">More details on the Amazon Forecast documentation page.</a>

In the next few cells, we ask for your Predictor arn and S3 location where to write the backtest export files.

In [ ]:
## Get user inputs for predictor arn and where to export the files

# TODO: Clear output of this cell which contains account#
# TODO: put back default value or user override value
# default = "arn:aws:forecast:us-west-2:123456789012:predictor/bike_demo_auto"
default_predictor_arn = "arn:aws:forecast:us-west-2:788825421177:predictor/nyc_taxi_new_with_indra"  
predictor_arn = getpass.getpass("weather predictor arn [enter to accept default]: ") \
                            or default_weather_predictor_arn

# default = "s3://bike-demo/forecasts/"
default_export_path = 's3://forecastdemogunjangarg/nyc_export/nyc_taxi_new_with_indra/'
export_path = input(f"weather predictor backtest export path [enter to accept default]:{default_weather_export_path}") \
                                or default_weather_export_path 



<br>
In the next few cells, we ask Amazon Forecast to export the Predictor backtest window forecasts via API.  The same could be done by <a href="https://aws.amazon.com/blogs/machine-learning/measuring-forecast-model-accuracy-to-optimize-your-business-objectives-with-amazon-forecast/, target='_blank' ">clicking the "Export backtest results" button on the Predictor page, as shown in the blog.</a>
<br>
<br>

In [ ]:

## Call CreatePredictorBacktestExportJob using predictor Arn and S3 export path

backtestExportJobName = 'bike_demo_forecasts'
backtest_export_job_response =forecast.create_predictor_backtest_export_job(PredictorBacktestExportJobName=backtestExportJobName,
                                                          PredictorArn=predictor_arn,
                                                          Destination= {
                                                              "S3Config" : {
                                                                 "Path":export_path,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          })

In [ ]:

# check for HTTPStatusCode 200

backtest_export_job_arn = backtest_export_job_response['PredictorBacktestExportJobArn']
backtest_export_job_response

In [ ]:

## CHECK STATUS OF YOUR EXPORT JOB - BACKTEST FORECASTS

status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_predictor_backtest_export_job(PredictorBacktestExportJobArn = \
                        backtest_export_job_response['PredictorBacktestExportJobArn'])['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

# Wait until you see "ACTIVE" below...
# This will take a while, go get a cup of tea now.  

The API steps you did above, could equivalently be done in the UI by clicking the "Export backtest results" button on the Predictor page.  You'll see export job details on the screen.

!["Export backtest results"](images/export_backtest_results.png "Export backtest results")

## Assemble and read predictor backtest files <a class="anchor" id="read"></a>

After Forecast Predictor Backtest Export step finishes, you will have a number of .part files within 2 separate folders.  The cell below concatenates all the .part files per folder into a single .csv file which can be saved to an S3 location of your choice. <br>

Make sure to change each section below with <b><i>your S3 locations</i></b>.
<br>

In [ ]:
## Get user inputs for where to find exported files

# TODO: put back default value or user override value
default_export_path = 's3://bike-demo/bike_share_open_data/export_files/bike_backtest_accuracies/'
export_path = input(f"weather predictor backtest export path [enter to accept default]:{default_export_path}") \
                                or default_export_path 

In [6]:
7 * 24

168

## Assemble and read backtest forecasts

After Forecast Predictor Backtest Export step finishes, you will have a number of .part files within 2 separate folders.  The cell below concatenates all the .part files per folder into a single .csv file which can be saved to an S3 location of your choice. <br>

Make sure to change each section below with <b><i>your S3 locations</i></b>.
<br>

In [ ]:
## ASSEMBLE S3 PATH TO INPUT FILES FROM PREVIOUSLY-ENTERED S3 path where you saved export files

# You should already have export_path from inputs above

# path to files is everything after BUCKET_NAME/, it should end in "/"
s3_path_to_files = export_path.split(BUCKET_NAME)[1][1:]
print(f"path to files: {s3_path_to_files}")
# s3://christy-forecast/open-data-analytics-taxi-trips/backtest_exports/forecasted-values/

In [ ]:


## Concat files in a way that works on any OS

if platform == "Windows":
    print("Your system is Windows")
    # copy part files locally
    try:
        shutil.rmtree("tempfcst")
    except OSError:
        pass
    os.makedirs("tempfcst")
    !aws s3 cp $from_files tempfcst/ --recursive --include "*.csv"

    # Concat .csv part files locally
    path = r'tempfcst'
    allFiles = glob.glob(path + "/*.csv")
    with open(temp_file, 'wb') as outfile:
        for i, fname in enumerate(allFiles):
            with open(fname, 'rb') as infile:
                if i != 0:
                    infile.readline()  # Throw away header on all but first file
                # Block copy rest of file from input to output without parsing
                shutil.copyfileobj(infile, outfile)
                print(fname + " has been imported.")

    # copy concatted local .csv file back to S3
    !aws s3 cp $temp_file $to_file_forecasts
    
elif platform == "Linux" or platform == "Darwin" :
    print("Your system is Linux")
    # copy part files locally
    !mkdir -p tempfcst
    !rm -rf tempfcst/*
    !aws s3 cp $from_files tempfcst/ --recursive --include "*.csv"

    # Concat .csv part files locally
    !touch $temp_file
    !cat tempfcst/*csv > $temp_file

    # copy concatted local .csv file back to S3
    !aws s3 cp $temp_file $to_file_forecasts
else:
    print("Unidentified operating system")

In [ ]:

## READ THE ACCURACIES FILE

accuracy_df = pd.read_csv(to_file_accuracies)

# keep only data rows
print(accuracy_df.shape)
accuracy_df = accuracy_df.loc[(accuracy_df.backtestwindow_start_time != "backtestwindow_start_time"), :].copy()
print(accuracy_df.shape)
accuracy_df.drop_duplicates(inplace=True)
print(accuracy_df.shape)

# correct data types
accuracy_df.item_id = accuracy_df.item_id.astype(str)
accuracy_df['backtestwindow_start_time'] = pd.to_datetime(accuracy_df['backtestwindow_start_time']
                                                 , format="%Y-%m-%dT%H:%M:%S", errors='coerce')
accuracy_df['backtestwindow_end_time'] = pd.to_datetime(accuracy_df['backtestwindow_end_time']
                                                 , format="%Y-%m-%dT%H:%M:%S", errors='coerce')
# convert UTC timestamp to timezone unaware
accuracy_df['backtestwindow_start_time'] = accuracy_df.backtestwindow_start_time.dt.tz_localize(None)
accuracy_df['backtestwindow_end_time'] = accuracy_df.backtestwindow_end_time.dt.tz_localize(None)

# correct dtypes
for q in accuracy_df.iloc[:, -4:].columns:
    accuracy_df[q] = pd.to_numeric(accuracy_df[q], errors='coerce')

# check
num_items = len(accuracy_df['item_id'].value_counts(normalize=True, dropna=False))
print(f"Num items: {num_items}")
print("Backtest Window Start Dates")
print(accuracy_df.backtestwindow_start_time.unique())

print(accuracy_df.dtypes)
accuracy_df.sample(5)

In [ ]:

## READ THE FORECASTS FILE

df = pd.read_csv(to_file_forecasts, low_memory=False)

# correct data types
df.item_id = df.item_id.astype(str)
df.target_value = pd.to_numeric(df.target_value, errors='coerce')
df.timestamp = pd.to_datetime(df.timestamp
                                                 , format="%Y-%m-%dT%H:%M:%S", errors='coerce')
df['backtestwindow_start_time'] = pd.to_datetime(df['backtestwindow_start_time']
                                                 , format="%Y-%m-%dT%H:%M:%S", errors='coerce')
df['backtestwindow_end_time'] = pd.to_datetime(df['backtestwindow_end_time']
                                                 , format="%Y-%m-%dT%H:%M:%S", errors='coerce')
# convert UTC timestamp to timezone unaware
df.timestamp = df.timestamp.dt.tz_localize(None)

# drop duplicates
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)

# check
num_items = len(df['item_id'].value_counts(normalize=True, dropna=False))
print(f"Num items: {num_items}")
print()
print("Backtest Window Start Dates")
print(df.backtestwindow_start_time.unique())

print(df.dtypes)
df.sample(5)

## Demo using the item-level forecast files <a class="anchor" id="demo"></a>

The rest of this notebook will focus on how to use the item-level forecasts from the Predictor backtest windows. 
<br>

#### Get quantile columns

In [ ]:
# Map column names in your data to expected key words
item_id = "item_id"
target_value = "target_value"
timestamp = "timestamp"
location_id = "item_id"

In [ ]:
# target = "target_value"
# set predictor dimensions from forecast df
predictor_cols = ['item_id', 'timestamp', 'rest_no', 'backtestwindow_start_time', 'backtestwindow_end_time']
# exclude cols to automatically find quantiles
exclude_cols = predictor_cols.copy()
exclude_cols.append(target_value)

# get quantile columns from forecast dataframe
quantile_cols = [c for c in df.columns if c not in exclude_cols] 
num_quantiles = len(quantile_cols)
print(f"num quantiles: {num_quantiles}")
quantile_cols

In [ ]:
# correct data types
for q in quantile_cols:
    df[q] = pd.to_numeric(df[q], errors='coerce')

print(df.dtypes)
df.sample(5)

#### Before calling error calcs, truncate negative actuals and predictions to 0
If you are not expecting negatives, such as for counts

In [ ]:

### Before calling error calcs, truncate negative actuals and predictions to 0

df_eligible = df.copy()
df_eligible = truncate_negatives_to_zero(df_eligible
                                         , target_value_col=target_value
                                         , quantile_cols=quantile_cols)


In [ ]:
# Add day of week for convenience
df_eligible['day_of_week'] = df_eligible.timestamp.dt.day_name()
print(df_eligible.day_of_week.value_counts())

# Add window number for convenience
windows = df_eligible.backtestwindow_start_time.value_counts().rename_axis('backtestwindow_start_time').reset_index(name='count')
windows.sort_values('backtestwindow_start_time', inplace=True)
windows.reset_index(inplace=True, drop=True)
windows.drop('count', axis=1, inplace=True)
windows['window'] = windows.index + 1

print(df_eligible.shape)
df_eligible = df_eligible.merge(windows, how="left", on="backtestwindow_start_time")
print(df_eligible.shape)
df_eligible.sample(5)

## Visualizations of Backtest Windows

Below is 1 chart per item, for 5 random items in the "fast" item group.  Y-axis is Actuals and color-coded Forecasts at each quantile.  X-axis is time, starting from the first Backtest Window and ending with the last Backtest Window.
<br>

In [ ]:

## get an x-range of dates of your data

print(df_eligible.backtestwindow_start_time.min())
print(df_eligible.backtestwindow_start_time.max())

x = pd.date_range(start=df_eligible.backtestwindow_start_time.min()
                    , end=df_eligible.backtestwindow_start_time.max() + timedelta(days=1), freq='D')
x = list(x)
x

In [ ]:

# Select random "fast" items
# random_items = df_eligible.loc[(df_eligible.velocity=="fast"), ['item_id']].copy()
# random_items = random_items.item_id.value_counts(dropna=False).index.tolist()
# random_items = random.sample(random_items, 5)
# print(len(random_items))

# instead of random fast, choose some fixed examples
random_items = ["31519", "31505", "31519", "31639", "31623"]

# gather data for plotting
forecasts = df_eligible.iloc[:, -len(quantile_cols)-2:-2]
dimension_cols = df_eligible[[item_id, timestamp, target_value]]
temp = pd.concat([dimension_cols, forecasts], axis=1)
print(temp.shape)
# rename "target_value" to "actual_value" for clearer viz
temp.rename(columns={'target_value':'actual_value'}, inplace=True)
temp = temp.groupby([timestamp, item_id]).sum()
temp.reset_index(inplace=True)
temp.set_index(timestamp, inplace=True)
print(temp.shape)
temp.head()

In [ ]:

# Visualize items
np.warnings.filterwarnings('ignore')  
fig, axs = plt.subplots(len(random_items), 1, figsize=(15, 15), sharex=True)
# axx = axs.ravel()


for i in range(len(random_items)):
    
    item = random_items[i]
    zoomed = temp.loc[(temp[item_id]==item), :]

    zoomed[['actual_value']].plot(ax=axs[i], color='k')
    colors = ['mediumpurple', 'orange', 'deepskyblue']
    
    for j in range(len(quantile_cols)):
        quantile = quantile_cols[j]
        zoomed[[quantile]].plot(ax=axs[i], color=colors[j])
            
    axs[i].set_title(f"Item_id={item}")
    axs[i].set_xlabel("Time")    #date
    axs[i].set_ylabel("Hourly demand")   
    axs[i].grid(axis='x')
    axs[i].set_xticks(x[0:])
    axs[i].set_xticklabels([str(dt.date())[0:11] for dt in x[0:]])



In [ ]:

# Visualize items - zoom in to see hours
np.warnings.filterwarnings('ignore')  
fig, axs = plt.subplots(len(random_items), 1, figsize=(15, 15), sharex=True)


for i in range(len(random_items)):
    
    item = random_items[i]
    zoomed = temp.loc[(temp[item_id]==item), :]
    zoomed = zoomed['2017-06-20':'2017-06-20']

    zoomed[['actual_value']].plot(ax=axs[i], color='k')    
    colors = ['mediumpurple', 'orange', 'deepskyblue']
    
    for j in range(len(quantile_cols)):
        quantile = quantile_cols[j]
        zoomed[[quantile]].plot(ax=axs[i], color=colors[j])
            
    axs[i].set_title(f"Item_id={item}")
    axs[i].set_xlabel("Time")    #date
    axs[i].set_ylabel("Hourly demand")   
    axs[i].grid(which='minor', axis='x')


Above, peak hours for bicycle rental appear to be between 5-9am and 3-9pm.  For a real customer study, we really should select more data to verify the peak hours...
<br>

## Demo custom item-level accuracy 

An example customer metric request might be - tell me MAPE for my top-selling items, during peak hours, and tell me this before deploying the Predictor.  Also, please use this MAPE formula.  MAPE = sum( |yhat - y| / |y| ).
<br>
<br>
To tackle this, we'll use the raw actuals, forecasts we just exported from the Predictor backtest windows.  Steps to calculate:
<ul>
    <li>First, we'll segment the items into "fast" and "slow" categories, depending on how much demand they have. </li>
    <li>Then we'll calculate a custom accuracy MAPE for each group of items.</li>  
    </ul>


#### Get "fast" vs "slow" items

In [ ]:

## CALCULATE DEMAND VELOCITY OF ITEMS

# categorize items
fast_moving_items, slow_moving_items = get_fast_slow_moving_items_all(df_eligible, timestamp, target_value, item_id)

# assign item velocity
df_eligible['velocity'] = "slow"
df_eligible.loc[(df_eligible.item_id.isin(fast_moving_items)), 'velocity'] = 'fast'

# checkit
print(df_eligible.velocity.value_counts(normalize=True, dropna=False))
df_eligible.sample(5)

In [ ]:

## Display breakdown: how many fast vs slow-moving items

total_items_cnt = len(fast_moving_items) + len(slow_moving_items)
print(f"number of fast moving items: {len(fast_moving_items)}, ratio:{len(fast_moving_items) / total_items_cnt}")

print(f"number of slow moving items: {len(slow_moving_items)}, ratio: {len(slow_moving_items) / total_items_cnt}")


#### Restrict to just peak hours
Assume peak hours for bicycle rental are Weekdays between 5-9am and 3-9pm.

In [ ]:

## Add peak hour flags

# add day of week and time of day
df_eligible['day_of_week'] = df_eligible[timestamp].dt.day_name()
df_eligible['time_of_day'] = df_eligible[timestamp].dt.time

# morning commute start and end
mc_s = pd.to_datetime('05:00:00').time()
mc_e = pd.to_datetime('10:00:00').time()

# evening commute start and end
ec_s = pd.to_datetime('15:00:00').time()
ec_e = pd.to_datetime('21:00:00').time()

# initialize flags to zero
df_eligible['peak_flag'] = 0
df_eligible['weekend_flag'] = 0

# add weekend flag
df_eligible['weekend_flag'] = df_eligible[timestamp].dt.dayofweek
df_eligible['weekend_flag'] = (df_eligible['weekend_flag'] >= 5).astype(int)

# add morning commute
df_eligible.loc[( (df_eligible.weekend_flag==0)
                    & ((df_eligible['time_of_day'] <= mc_e) 
                        & (df_eligible['time_of_day'] >= mc_s)) ), 'peak_flag'] = 1

# add evening commute
df_eligible.loc[( (df_eligible.weekend_flag==0)
                    & ((df_eligible['time_of_day'] <= ec_e) 
                        & (df_eligible['time_of_day'] >= ec_s)) ), 'peak_flag'] = 1

# check you did the right thing
# df_eligible.sample(70).sort_values('time_of_day')

In [ ]:

## Restrict evaluation to just peak hours

print(df_eligible.shape)
df_eligible = df_eligible.loc[(df_eligible.peak_flag==1), :].copy()
print(df_eligible.shape)
df_eligible.sample(3)

### Calculate custom metric MAPE per quantile for the "fast" item group <a class="anchor" id="mape"></a>

In [ ]:

### CALCULATE MAPE PER QUANTILE ACROSS ALL BACKTEST WINDOWS FOR FAST GROUPS OF ITEMS

from collections import defaultdict
mape_by_moving_fast = dict()
mape_fast = []

# FAST ITEMS
windows_list = list(windows.window)
for q in quantile_cols:
    quantile_list = []
    for w in windows_list:
        temp = df_eligible.loc[((df_eligible.velocity=="fast")
                       & (df_eligible.window==w)), [target_value, q]].copy()
        mape_by_moving_fast[q,w] = temp.apply(lambda row: calc_mape(row[target_value], row[q]), axis=1)
        
        quantile_list.append(mape_by_moving_fast[q,w])
    mape_fast.append(np.mean(quantile_list))
        
mape_fast = pd.DataFrame(mape_fast).T
mape_fast.columns = quantile_cols
mape_fast.index.name = 'MAPE'
print("Fast Item Custom MAPE per quantile")
mape_fast

In [ ]:
### LOOK UP STANDARD METRICS FOR THE GROUP OF "FAST" ITEMS

fast_metrics = accuracy_df.loc[(accuracy_df.item_id.isin(fast_moving_items)), :].copy()
# drop the summary row
fast_metrics = fast_metrics.loc[(fast_metrics.backtest_window != "Summary"), :].copy()
# calc mean of the standard metrics across 5 backtest windows
fast_metrics = fast_metrics.mean()
fast_metrics['item_id'] = "mean"

print("Fast Item Mean Standard Metrics per quantile")
fast_metrics

Note about "Accuracy".  Tech people tend to talk about errors since that is what is calculated.  Depending on your background, errors have different terms.
<ul>
    <li>Statisticians refer to errors as "Residuals". </li>
    <li>Machine learning folks refer to errors as "Loss". </li>
    <li>Business people tend to talk about "Accuracy", which is 100% - error rate. </li>
    </ul>

In [ ]:
### CONVERT CUSTOM MAPE TO "ACCURACY"

print("Fast Item Custom MAPE per quantile")
100 - mape_fast

In [ ]:
### CONVERT STANDARD ERROR METRICS FOR THE GROUP OF "FAST" ITEMS TO "ACCURACY"

print("Fast Item Accuracy per quantile")
(1.0 - fast_metrics) * 100.0

## Cleanup <a class="anchor" id="cleanup"></a>

In [ ]:
delete_backtest_export_job_response = \
    forecast.delete_predictor_backtest_export_job(PredictorBacktestExportJobArn = backtest_export_job_arn)
delete_backtest_export_job_response